In [ ]:
import pandas as pd

from serenity_sdk.client import load_local_config
from serenity_sdk.client import SerenityApiProvider, SerenityClient

%load_ext autoreload
%autoreload 2

config_id = 'athansor'  # Use your own `config_id`
config = load_local_config(config_id)

client = SerenityClient(config)
api = SerenityApiProvider(client)

Serenity supports multiple asset types to encompass instruments like tokens, fiat currency and derivatives. You can get a simple mapping of the type names and their descriptions via the API:

In [ ]:
asset_types = api.refdata().get_asset_types()
pd.DataFrame.from_dict(asset_types, orient='index', columns=['Description'])

A symbol authority is an organization that defines a symbology, a universe of symbols for assets. You can use different authorities like DAR or KAIKO for symbols, and then map them into symbols. This API call lets you get their names and descriptions:

In [ ]:
authorities = api.refdata().get_symbol_authorities()
pd.DataFrame.from_dict(authorities, orient='index', columns=['Description'])

Serenity's risk attribution and other tools can use sector taxonomies to create hierarchies of assets. Because this is extensible, they each have unique identifiers. This call lets you get you those identifiers by name, e.g. DATS:

In [ ]:
sector_taxonomies = api.refdata().get_sector_taxonomies()
pd.DataFrame.from_dict(sector_taxonomies, orient='index', columns=['ID'])

If you just want to browse the asset catalog, you can use this call to get all the asset records (in JSON format):

In [ ]:
asset_summaries = api.refdata().get_asset_summaries()
asset_summaries = [{key: value for key, value in summary.items() if key != 'xrefSymbols'} for summary in asset_summaries]
pd.json_normalize(asset_summaries)

Finally, the most common use for reference data is to map from your own system's identifiers to Serenity's identifiers. There is a helper class that does this for you called AssetMaster. Here are three different mappings using different symbologies:

In [53]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'BTC': 100,
    'ETH': 1000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')
positions = portfolio.to_asset_positions()
pd.json_normalize(positions)

,assetId,quantity
0,78e2e8e2-419d-4515-9b6a-3d5ff1448e89,100
1,95b89dfd-c5f6-41d6-83c6-a9d97e0ac361,1000


In [51]:
portfolio_raw = {
    'tok.btc.bitcoin': 100,
    'tok.eth.ethereum': 1000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='SERENITY')
positions = portfolio.to_asset_positions()
pd.json_normalize(positions)

,assetId,quantity
0,78e2e8e2-419d-4515-9b6a-3d5ff1448e89,100
1,95b89dfd-c5f6-41d6-83c6-a9d97e0ac361,1000


In [52]:
portfolio_raw = {
    'bitcoin': 100,
    'ethereum': 1000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='COINGECKO')
positions = portfolio.to_asset_positions()
pd.json_normalize(positions)

,assetId,quantity
0,78e2e8e2-419d-4515-9b6a-3d5ff1448e89,100
1,95b89dfd-c5f6-41d6-83c6-a9d97e0ac361,1000
